In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from enum import Enum
import re
import pandas as pd



In [2]:
# Set whether to display selium browser
VISIBLE = True

options = Options()
# If not visible do not display browser to user
if not VISIBLE: options.add_argument('--headless')

url = "https://goblin.bet/#/"
driver = webdriver.Firefox(options=options)
driver.get(url)




In [3]:
contButt = WebDriverWait(driver, timeout=30).until(lambda d: d.find_element(By.CLASS_NAME, "WelcomeStart.Left"))
contButt.click()

In [4]:
pageSrc = driver.page_source
soup = BeautifulSoup(pageSrc)

RIGHT = True
LEFT = False

def getLog(soup: BeautifulSoup):
    log = []
    # Find log
    logSoup = soup.find("div", {"class": "scrollhost BLogScroll"})

    # If print log true print out the log
    for lg in logSoup.find_all("div", {"class": "LogText"}):
        log.append(lg.text)
    # return the betting log
    return log

def getMoney(soup: BeautifulSoup):
    return soup.find('span', {"class": "BetsScore"}).text

def getCreatureSoup(soup: BeautifulSoup, side=LEFT):
    if side == LEFT:
        return soup.find("div", {"class": "Block Statsheet Left TeamRed"})
    return soup.find("div", {"class": "Block Statsheet Right TeamBlue"})
        

In [31]:
# Strips all alphabetic characters
def stripChrs(word):
    return re.sub("[^0-9]", '', word)

# Strips all nonalphanumeric characters in string
def stripInts(word):
    return re.sub(r'\W+', '', word)

# Finds a tag with a specfic class name
def findSpanClass(soup, className, all=False):
    if not all:
        return soup.find("span", {"class": className})
    return soup.find_all("span", {"class": className})


# Class containing all betting creature information 
class Creature:

    def __init__(self, soup: BeautifulSoup, side=LEFT):
        self.side = side
        self.soup = getCreatureSoup(soup, side)
        self.name = self.InitName(self.soup)
        self.desc, self.cr = self.InitDescStats(self.soup)
        self.InitStats()
        
    def InitName(self, soup: BeautifulSoup):
        return findSpanClass(soup, "SSName").text
    
    # Intializes description stats and cr rank
    def InitDescStats(self, soup: BeautifulSoup):
        cretInfo = None
        cr = None
        for i, tag in enumerate(soup.find_all('span', {'class': "SSInfo"})):
            if i == 0:
                cretInfo = tag.text
            if i == 1:
                cr = tag.text.split(' ')[1]
        return cretInfo, cr

    def getAction(self, action: BeautifulSoup):
        action = [findSpanClass(action, "ActName").text, findSpanClass(action, "ActDesc").text]
        return action

    # Initializes all stats in the creature sheet
    def InitStats(self):
        # Initialize stat categories
        self.stats = None
        self.immunities = None
        self.resists = None
        self.actions = None
        self.conditions = None
        self.wins = None

        # Gets list of all tags in the stats soup
        statsList =  self.soup.find("div", {"class": "SSStats"})
        # Go through every header value in the tags
        for head in statsList.find_all("span", {"class": "SSHeader"}):
            # Record the text of the header
            headTxt = stripInts(head.text)
            if headTxt == "ATTRIBUTES":
                self.stats = self.getAttributes(statsList)
            if headTxt == "WINS":
                self.wins = head.find_next_sibling('span').text.split(',')
            # If content is immunities
            if headTxt == "IMMUNE":
                # Store immunities using next tag
                self.immunities = [word.replace(' ', '') for word in head.find_next_sibling("span").text.split(',')]
            if headTxt == "RESIST":
                # Store resistances using next tag
                self.resists = [word.replace(' ', '') for word in head.find_next_sibling("span").text.split(',')]
            if headTxt == "ACTIONS":
                self.actions = []
                # Append each action to list
                self.InitActions(head)
            if headTxt == "CONDITIONS":
                # Find all conditions
                self.conditions =  self.getConditions(statsList)
            
    # Initializes possible actions of the 
    def InitActions(self, actionHead):
        try:
            child = actionHead
            for action in child.find_next_siblings("div"):
                self.actions.append(self.getAction(action))
            child = child.next
        except AttributeError:
            None

    
    def getConditions(self, statsList: BeautifulSoup):
        conditions = []
        # Highlighted condition
        for tag in findSpanClass(statsList, "Stat Small CanPop Feat"):
            conditions.append(tag.text)
            # Break as the next tag is redundant
            break
        # Loop through all remaining conditions
        for tag in findSpanClass(statsList, "Stat Small", all=True):
            conditions.append(tag.text)

        return conditions


    # Copies attributes tag
    def getAttributes(self, statsList: BeautifulSoup):
        stats = {"STR": None, "DEX": None, "CON": None, "INT": None, "WIS": None, "CHA": None}
        for i, stat in enumerate(statsList.find_all("span", {"class": "Stat"})):
            if i == 0:
                stats["STR"] = stripChrs(stat.text)
            elif i == 1:
                stats["DEX"] = stripChrs(stat.text)
            elif i == 2:
                stats["CON"] = stripChrs(stat.text)
            elif i == 3:
                stats["INT"] = stripChrs(stat.text)
            elif i == 4:
                stats["WIS"] = stripChrs(stat.text)
            elif i == 5:
                stats["CHA"] = stripChrs(stat.text)
            elif i == 6:
                hpVals = [val for val in re.split("/", stat.text)]
                stats["HP"] = stripChrs(hpVals[0])
                stats["HPMax"] = stripChrs(hpVals[1])
            elif i == 7:
                stats["AC"] = stripChrs(stat.text)
            elif i == 8:
                stats["SPD"] = stripChrs(stat.text)
        return stats

    
    def getInfoStrings(self, list):
        if list == None:
            return None
        return ",".join(list)

    # Converts creature action 2d array into a 1d list of strings
    def getActionStrings(self):
        return "\t".join(["".join(action) for action in self.actions])

    # Returns all information in Creature class as a dictionary
    def getDataLoader(self):
        # Initialize creature dictionary
        cretDict = {"name": self.name, "desc": self.desc, "cr": self.cr, "immunities": self.getInfoStrings(self.immunities), 
                    "resists": self.getInfoStrings(self.resists), "conditions": self.getInfoStrings(self.conditions), "wins": self.getInfoStrings(self.wins), "actions": self.getActionStrings(),
                    "hp": self.stats["HPMax"], "str": self.stats["STR"], "dex": self.stats["DEX"], "con": self.stats["CON"], 
                    "int": self.stats["INT"], "wis": self.stats["WIS"], "cha": self.stats["CHA"], "ac": self.stats["AC"],
                    "spd": self.stats["SPD"]}

        return cretDict

                
soup = BeautifulSoup(driver.page_source)
cret = Creature(soup, RIGHT)


In [32]:
def creatureLog(cret: Creature):
    print(f"Name: {cret.name}")
    print(f"Description: {cret.desc}")
    print(f"CR: {cret.cr}")
    print("Side: {}".format("Right" if cret.side == RIGHT else "Left"))
    print(f"Wins: {cret.wins}")
    print(f"Stats: {cret.stats}")
    print(f"Actions: {cret.actions}")
    print(f"Resistances: {cret.resists}")
    print(f"Immunitises: {cret.immunities}")
    print(f"Conditions: {cret.conditions}")
creatureLog(cret)


Name: Wyvern
Description: Large Dragon Unaligned
CR: 6
Side: Right
Wins: ['Green Hag']
Stats: {'STR': '19', 'DEX': '10', 'CON': '16', 'INT': '5', 'WIS': '12', 'CHA': '6', 'HP': '63', 'HPMax': '63', 'AC': '13', 'SPD': '80'}
Actions: [['Multiattack: ', '1x (Bite, Claws), Stinger'], ['Bite: ', '+6, 10ft, 2d6+4 Piercing'], ['Claws: ', '+6, 2d8+4 Slashing'], ['Stinger: ', '+6, 10ft, 2d6+4 Piercing, 7d6 Thunder (CON Save)']]
Resistances: None
Immunitises: None
Conditions: ['Wild Element']


In [41]:
soup = BeautifulSoup(driver.page_source)
cret = Creature(soup, RIGHT)
cret.actions
print(cret.getActionStrings())

dfTest = pd.DataFrame(cret.getDataLoader(), index=[0])
dfTest.head()

Bite: +5, 3d6+3 Thunder	Claw: +5, 1d6+3 Thunder	Multiattack: 3 Claw, Bite


,name,desc,cr,immunities,resists,conditions,wins,actions,hp,str,dex,con,int,wis,cha,ac,spd
0,Xorn,Medium Elemental Neutral,5,None,"Piercing,Slashing,Thunder","Thunder Attuned,Grappled & Restrained","Shambling Mound, Wyvern","Bite: +5, 3d6+3 Thunder\tClaw: +5, 1d6+3 Thund...",38,17,10,22,11,10,11,19,20


In [75]:
dfWin = pd.read_csv("./Winners.csv", index_col=0)
dfLose = pd.read_csv("./Losers.csv", index_col=0)

dfWin.head()

,name,desc,cr,immunities,resists,conditions,wins,actions,hp,str,dex,con,int,wis,cha,ac,spd
0,Wraith of Ogre Power,Medium Undead Neutral Evil,5,"Necrotic,Poison,Charmed,Exhaustion,Grappled,Pa...","Acid,Cold,Fire,Lightning,Thunder,Bludgeoning,P...",Gauntlets of Ogre Power,"Werebear, Air Elemental, Troll, Roper, CR 5","Life Drain: +11, 4d8+9 Necrotic & Life Drain",39,19,16,16,12,14,15,13,60
1,Oni,Large Giant Lawful Evil,7,NaN,NaN,"Death Defying,Regeneration",NaN,"Glaive: +6, 10ft, 2d10+4 Slashing\tMultiattack...",55,19,11,16,14,12,15,16,30
2,Giant Ape,Huge Beast Unaligned,7,NaN,NaN,Shielded,NaN,"Fist: +8, 10ft, 3d10+6 Bludgeoning\tRock: +8, ...",69,23,14,18,7,12,7,12,40
3,Young Brass Dragon,Large Dragon Chaotic Good,6,Fire,NaN,Reliable Damage,NaN,"Multiattack: 10ft, Bite, 2 Claw\tBite: +6, 10f...",61,19,10,17,12,11,15,17,80
4,Fire Elemental,Large Elemental Neutral,5,"Fire,Poison,Exhaustion,Grappled,Paralyzed,Petr...","Bludgeoning,Piercing,Slashing","+2 CHA,Heated Body",NaN,"Touch: +5, 2d6+3 Fire, On Fire\tMultiattack: F...",46,10,17,16,6,10,9,13,50


In [76]:
import time


prevLog = []

while True:# Get Soup

    soup = BeautifulSoup(driver.page_source)
    
    if soup != None:
        # Gets current log
        log = getLog(soup)
        if len(log) <= 0:
            continue
        currentEntry = log[0]
        

        # Check if logIsEqual
        if prevLog != log and prevLog != None and log != None:
            # Prints current entry
            print(currentEntry)
            # Check for winner
            if "eliminated!" in currentEntry:
                # Initialize creatures
                cretR = Creature(soup, side=RIGHT)
                cretL = Creature(soup)

                if cretR.name + " wins!" in currentEntry:
                    dfWin = pd.concat([dfWin, pd.DataFrame(cretR.getDataLoader(), index=[0])], ignore_index=True)
                    dfLose = pd.concat([dfLose, pd.DataFrame(cretL.getDataLoader(), index=[0])], ignore_index=True)
                elif cretL.name + " wins!" in currentEntry:
                    dfWin = pd.concat([dfWin, pd.DataFrame(cretL.getDataLoader(), index=[0])], ignore_index=True)
                    dfLose = pd.concat([dfLose, pd.DataFrame(cretR.getDataLoader(), index=[0])], ignore_index=True)
                else:
                    print("Expected creature name, got nothing")
        prevLog = log

        
        

            
    time.sleep(0.9)


+ The Glabrezu's Fist misses!
+ Dullahan uses Baleful Glare.
+ Glabrezu becomes Baleful Terror (13 vs DC:15).
+ Dullahan resists Stunned (CON) : SUCCESS
+ Glabrezu attacks Dullahan with a Pincer.
+ The Glabrezu's Pincer misses!


KeyboardInterrupt: 

In [67]:
dfWin.to_csv("./Winners.csv")
dfLose.to_csv("./Losers.csv")